# Quantifying uncertainty due to heterogeneous material fields

In this tutorial we'll have a look at prescribing heterogenous material parameter fields using 4C in order to do an uncertainty quantification on the resulting Cauchy stresses.

# The 4C model

In this example, we'll have a look at a nonlinear solid mechanics problem. The momentum equation is given by

$$\ddot{\boldsymbol{u}} = \nabla \cdot \boldsymbol{\sigma} + \boldsymbol{b} \text{ in } \Omega \times T$$

where $u$ are the displacements, $\boldsymbol{\sigma}$ the Cauchy stress and $b$ volumetric body forces. For the examples, $\boldsymbol{b}=\boldsymbol{0}$, and we prescribe Dirichlet boundary conditions
$$ u_x = 2.5 \frac{t}{T_s} \text{ on } \Gamma_r = \{x=12.5\} \times T$$
$$ u_y = 2.5 \frac{t}{T_s} \text{ on } \Gamma_t = \{y=12.5\} \times T$$
$$ u_y = 0 \text{ on } \Gamma_b = \{y=-12.5\} \times T$$
$$ u_x = 0 \text{ on } \Gamma_l = \{x=-12.5\} \times T$$

where $T_s = 40$ and $T=[0, T_s]$. For $t=0$ we set $\boldsymbol{u}=\boldsymbol{0}$.

The domain $\Omega$ is a two-dimensional hyperelastic isochoric membrane, assuming plane stress with strain energy function

$$\Psi = \frac{\mu}{2} (J^{-2/3}I_{\boldsymbol{C}}-3)$$

where $\mu$ is the shear modulus, $I_{\boldsymbol{C}}$ is the first invariant of the Cauchy-Green tensor and $J$ the determinant of the deformation gradient. 

The problem is discretized in space using linear hexahedral finite elements and finite differences in time.

> Note: This tutorial is based on [this](https://github.com/queens-py/queens/blob/main/tests/input_files/third_party/fourc/coarse_plate_dirichlet_template.4C.yaml) QUEENS test.

# Uncertainty quantification

The goal of this example is to compute the expectation of the Cauchy stress tensor for $t=T_s$

$$\overline{\boldsymbol{\sigma}}=\int \boldsymbol{\sigma}(\mu)p(\mu)d\mu$$

where the shear modulus $\mu$ is a random field. This scenario lends itself to study the effect of the heterogenous constitutive material field on the Cauchy stress.

Using the law of the unconscious statistician, we can reformulate the problem as
$$\overline{\boldsymbol{\sigma}}=\int \boldsymbol{\sigma}(\mu)p(\mu)d\mu=\int \boldsymbol{\sigma}(\mu(\boldsymbol{\theta}))p(\boldsymbol{\theta})d\boldsymbol{\theta}$$

where $\theta$ are parameters describing the heterogenous material field $\mu(\theta)$.

## The random field

Let's define a random field:
$$\mu(\theta) = 0.1+\exp\left(-0.5\frac{(x-\theta_1)^2 + (y-\theta_2)^2}{16}\right)$$

We set the distribution of the latent variable to be a multivariate Normal $p(\boldsymbol{\theta})=\mathcal{N}(\boldsymbol{\theta}|\boldsymbol{0}, 5\boldsymbol{I})$ with mean value $\boldsymbol{0}$ and a variance of $5$.

In [ ]:
import numpy as np


def random_field_function(theta, positions):
    diff = positions.copy()[:, :2]

    diff[:, 0] = diff[:, 0] - theta[0]
    diff[:, 1] = diff[:, 1] - theta[1]
    return 0.1 + np.exp(-0.5 * (np.sum(diff**2, axis=1)) / 16)


from queens.distributions import Normal

theta_distribution = Normal(np.array([0, 0]), 5 * np.eye(2))

# Monte Carlo integration

Since the integral above can not be computed analytically, we employ Monte Carlo integration

$$\overline{\boldsymbol{\sigma}}=\int \boldsymbol{\sigma}(\mu(\boldsymbol{\theta}))p(\boldsymbol{\theta})d\boldsymbol{\theta} \approx \frac{1}{N} \sum_{s=0}^N \boldsymbol{\sigma}(\mu(\boldsymbol{\theta}^{(s)}))$$

where $\boldsymbol{\theta}^{(s)}$ are independent and identically distributed samples drawn from $p(\boldsymbol{\theta})$. For this examples we'll set $N=100$, so 100 4C runs will be done per QUEENS run.

> Note: If you are using the docker container, plotting with pyvista might not work. In that case, please ask your peers in the group to show and discuss their results. You can also refer to the pdf solution file for figures.

In [ ]:
# Let's get the domain from the 4C input file
from queens_interfaces.fourc.random_material_preprocessor import (
    create_jinja_json_template,
)
import pyvista as pv

# Open the mesh for the random field
mesh = pv.read("membrane_20.e")[0][0]

# Compute the cell centers
centers = mesh.cell_centers().points

# Construct the RF information
mu_rf_parameters = {
    "coords": centers,
    "keys": [f"MUE_{i}" for i in range(1, 1 + len(centers))],
}

# Contains the random field values
material_file_template = "material.json"

# Create the material file for 4C
create_jinja_json_template(
    "MUE",
    np.arange(1, len(centers) + 1),
    mu_rf_parameters["keys"],
    material_file_template,
)


from utils.random_field import CustomRandomField

random_field = CustomRandomField(
    mu_rf_parameters,  # Parameter names and coordinates
    theta_distribution,  # Latent variables
    random_field_function,  # Expansion, transformation from theta to mu
)

Now that we defined a random field, let's look at some samples!

In [ ]:
# Let's plot some samples

from utils.plot_input_random_field import plot_field

latent_samples = random_field.draw(3)
mu_samples = random_field.expanded_representation(latent_samples)

plotter = pv.Plotter(shape=(1, 3))

for i, mu_s in enumerate(mu_samples):
    plotter.subplot(0, i)
    plot_field(mu_s, plotter, f"Sample field mu {i}\n")

plotter.show()

As we can see, we set the material parameter to be constant in each element. We can think about this random field of a domain that has some type of defect where the mechanical properties are different.

In [ ]:
# Let's do Monte Carlo integration

from queens.parameters import Parameters
from queens.data_processors import PvdFile
from queens.schedulers import Local
from queens_interfaces.fourc.driver import Fourc
from queens.models import Simulation
from queens.iterators import MonteCarlo
from queens.global_settings import GlobalSettings
from queens.main import run_iterator
from queens.utils.io import load_result
import pathlib

# Set the paths docker
home = pathlib.Path("/home/user")
fourc_executable = "/home/user/4C/build/4C"
input_template = "coarse_plate_dirichlet_template_docker.4C.yaml"

# Set the paths for the virtual machine
home = pathlib.Path("/home/participant")
fourc_executable = "/home/participant/4C/build/release/4C"
input_template = "coarse_plate_dirichlet_template.4C.yaml"

if __name__ == "__main__":
    pathlib.Path("monte_carlo_4C").mkdir(exist_ok=True)
    with GlobalSettings(
        "monte_carlo_random_field_4C", output_dir="monte_carlo_4C"
    ) as gs:
        parameters = Parameters(MUE=random_field)

        # Extract the Cauchy stresses at the last time step
        data_processor = PvdFile(
            field_name="element_cauchy_stresses_xyz",
            file_name_identifier="output-structure.pvd",
            file_options_dict={},
            point_data=False,
            time_steps=[-1],
        )

        # How to run 4C
        driver = Fourc(
            parameters=parameters,
            input_templates={
                "input_file": input_template,  # Input file for 4C
                "material_file": material_file_template,  # File containing the random field
            },
            executable=fourc_executable,
            data_processor=data_processor,
        )

        # Schedule parallel simulations
        scheduler = Local(num_procs=1, num_jobs=4, experiment_name=gs.experiment_name)

        # our model
        model = Simulation(scheduler=scheduler, driver=driver)
        iterator = MonteCarlo(
            seed=1,
            num_samples=100,
            result_description={"write_results": True, "plot_results": False},
            model=model,
            parameters=parameters,
            global_settings=gs,
        )

        run_iterator(iterator, gs)

        results = load_result(gs.result_file("pickle"))

In [ ]:
# Let's plot the mean Cauchy stresses

input_samples = results["input_data"]
cauchy_stresses = results["raw_output_data"]["result"]


plotter = pv.Plotter(shape=(1, 3))

plotter.subplot(0, 0)
plot_field(
    color_bar_title=f"Mean Cauchy Stress xx\n",
    field=results["mean"][:, 0],
    plotter=plotter,
)

plotter.subplot(0, 1)
plot_field(
    color_bar_title=f"Mean Cauchy Stress yy\n",
    field=results["mean"][:, 1],
    plotter=plotter,
)

plotter.subplot(0, 2)
plot_field(
    color_bar_title=f"Mean Cauchy Stress xy\n",
    field=results["mean"][:, 3],
    plotter=plotter,
)

plotter.show()

In [ ]:
# Let's have a look at some outputs

plotter = pv.Plotter(shape=(4, 3))

job_ids = [0, 50, 99]
for i, job_id in enumerate(job_ids):
    plotter.subplot(0, i)
    plot_field(
        color_bar_title=f"Sample {job_id}: Shear modulus",
        field=random_field.expanded_representation(input_samples[job_id, :]),
        plotter=plotter,
    )
    plotter.subplot(1, i)
    plot_field(
        color_bar_title=f"Sample {job_id} Cauchy Stress xx",
        field=cauchy_stresses[job_id, :][:, 0],
        plotter=plotter,
    )
    plotter.subplot(2, i)
    plot_field(
        color_bar_title=f"Job {job_id} Cauchy Stress yy",
        field=cauchy_stresses[job_id, :][:, 1],
        plotter=plotter,
    )
    plotter.subplot(3, i)
    plot_field(
        color_bar_title=f"Job {job_id} Cauchy Stress xy",
        field=cauchy_stresses[job_id, :][:, 3],
        plotter=plotter,
    )
plotter.show()

Nice, we started 4C using QUEENS and prescribed a heterogeneous constitutive parameters! Note that compared to the previous examples, we only had to adapt the parameters! From driver to scheduler to model to iterator, nothing changed, highlighting the modularity of QUEENS.

# Gaussian random fields

There are various ways of defining a random field, a common one being Gaussian random field (also known as [Gaussian process](https://en.wikipedia.org/wiki/Gaussian_process))! The main aspect of Gaussian random fields $f(x,y)$, is that at every location $x,y$ of the domain, the random field is normally distributed:

$f(x,y) \sim \mathcal{N}(f|\mu_f(x,y),K((x,y), (x',y')))$

Here $\mu_f(x,y)$ is the mean value function and $K$ the covariance function. The latter one, describes the properties of random field, such as smoothness, lengthscales, etc.

Gaussian random fields can be sampled using the [Kosambi–Karhunen–Loève](https://en.wikipedia.org/wiki/Kosambi%E2%80%93Karhunen%E2%80%93Lo%C3%A8ve_theorem) theorem:

$$f^{(s)}(x,y) = \mu_f(x,y)+\sum_{k=1}^{\infty} \sqrt{\lambda_k} \xi_k^{(s)} e_k(x,y)$$

where $\lambda_k$ and $e_k$ are the $k$-th eigenvalues and eigenfunctions of $K$, where $\boldsymbol{\xi}^{(s)}$ are samples of Gaussian random variables with zero mean and identity covariance matrix. This approach is similar to [proper orthogonal decomposition](https://en.wikipedia.org/wiki/Proper_orthogonal_decomposition).

For these, we truncate this series at some value $k_{t}$. We select the covariance function

$$K((x,y),(x',y')) = \sigma_G^2 \exp\left(-\frac{(x-x')^2 + (y-y')^2}{2l^2}\right)$$

where we set the length scale $l=8.0$ and the output variance $\sigma_G^2 = 0.03$. The mean function is assumed constant as $\mu_f(x,y)=0.25$. The number of eigenfunctions $k_t$ is chosen such that the explained variance equals 0.95, i.e. $\frac{\sum_{k=1}^{k_t} \lambda_k}{\sum_{j=1}^{\infty} \lambda_j} \approx 0.95$


*Enough theory, let's look at some samples*

In [ ]:
from queens.parameters.random_fields import KarhunenLoeve

np.random.seed(42)

random_field = KarhunenLoeve(
    corr_length=8.0,
    std=0.03,
    mean=0.25,
    explained_variance=0.95,
    coords=mu_rf_parameters,
)

latent_samples = random_field.draw(3)
mu_samples = random_field.expanded_representation(latent_samples)

plotter = pv.Plotter(shape=(1, 3))

for i, mu_s in enumerate(mu_samples):
    plotter.subplot(0, i)
    plot_field(mu_s, plotter, f"Sample field mu {i}\n")

plotter.show()

You can see how the samples are distinct from the previous examples. They are more 'wiggly', yet still one can see a correlation structure. This is precisely what the length scale and variance does!

Try it out yourself, decrease/increase the parameters and look at the samples.

What do these parameters do?

<details>

<summary>Answer</summary>

According to the [The Kernel Cookbook by David Duvenaud](https://www.cs.toronto.edu/~duvenaud/cookbook/):

- The lengthscale determines the length of the 'wiggles' in your function. In general, you won't be able to extrapolate more than $l$
units away from your data.
- The output variance $\sigma_G^2$ determines the average distance of your function away from its mean. Every kernel has this parameter out in front; it's just a scale factor.
</details>

Look at the cutoff value $k_t$ (hint: `random_field.dimension`). How does this change depending on the length scale?

<details>

<summary>Answer</summary>

With smaller length scales, the 'wiggle'-frequency increases, hence more components are needed to explain the desired variance in eigenvalues of 95%.
</details>



Nice, now let us repeat the Monte Carlo experiment using the Gaussian random field describe above!

In [ ]:
if __name__ == "__main__":
    with GlobalSettings(
        "monte_carlo_random_field_4C", output_dir="monte_carlo_4C"
    ) as gs:
        parameters = Parameters(MUE=random_field)

        # Extract the Cauchy stresses at the last time step
        data_processor = PvdFile(
            field_name="element_cauchy_stresses_xyz",
            file_name_identifier="output-structure.pvd",
            file_options_dict={},
            point_data=False,
            time_steps=[-1],
            files_to_be_deleted_regex_lst=[
                "output.control",
                "*.mesh.s0",
                "output-vtk-files/*",
            ],  # These files are deleted
        )

        # How to run 4C
        driver = Fourc(
            parameters=parameters,
            input_templates={
                "input_file": input_template,  # Input file for 4C
                "material_file": material_file_template,  # File containing the random field
            },
            executable=fourc_executable,
            data_processor=data_processor,
        )

        # Schedule parallel simulations
        scheduler = Local(num_procs=1, num_jobs=4, experiment_name=gs.experiment_name)

        # our model
        model = Simulation(scheduler=scheduler, driver=driver)
        iterator = MonteCarlo(
            seed=1,
            num_samples=100,
            result_description={"write_results": True, "plot_results": False},
            model=model,
            parameters=parameters,
            global_settings=gs,
        )

        run_iterator(iterator, gs)

        results = load_result(gs.result_file("pickle"))

# Let's plot the mean Cauchy stresses

input_samples = results["input_data"]

plotter = pv.Plotter(shape=(1, 3))

plotter.subplot(0, 0)
plot_field(
    color_bar_title=f"Mean Cauchy Stress xx\n",
    field=results["mean"][:, 0],
    plotter=plotter,
)

plotter.subplot(0, 1)
plot_field(
    color_bar_title=f"Mean Cauchy Stress yy\n",
    field=results["mean"][:, 1],
    plotter=plotter,
)

plotter.subplot(0, 2)
plot_field(
    color_bar_title=f"Mean Cauchy Stress xy\n",
    field=results["mean"][:, 3],
    plotter=plotter,
)

plotter.show()

# Why QUEENS?

- Hide complexity in handling complex simulation codes (same as in the last example)
- QUEENS hides also the complexity of working with random fields:
  - Monte Carlo iterator does not care that it is a random field
  - We can still use the native 4C driver even though we now have 2 input files (one for the simulation parameters, one for the random field)
  
## Let's play around

Let your creativity flow and try out stuff.

### Inspirations
- Create your own random field definition, maybe a sine wave? (Keep in mind the parameter has to remain positive!)
- Look some simulation output of the Monte Carlo run (change to the data processor attribute `files_to_be_deleted_regex_lst` to `None`)
- Rerun the Monte Carlo analysis with different length scales
- Rerun the Monte Carlo analysis with different number of samples
- Plot the variance estimate from the Monte Carlo run
- Change length scale and look at the Monte Carlo mean value for multiple runs with the same number of samples